# シュミレータ作り
目標: 分かりやすいシュミレータをnumpyで作る。

In [110]:
import numpy as np
import math
import scipy.sparse as sp

素晴らしいシュミレータを見つけたのでこれを参考にして作っていこうと思う。  
[シュミレータを作りながら量子コンピュータの仕組みを理解する](https://qiita.com/ohtaman/items/e121a2b0b4525b85b54d)

# 量子状態クラス

In [111]:
class Qubits(object):
    def __init__(self, n_bits):
        """
        Arguments:
            n_bits (int): 量子ビット数
        """
        self.n_bits   = n_bits                                     # 量子ビット数
        self.n_states = 2**n_bits                                  # 行数
        self._amp     = np.zeros(self.n_states, dtype=np.complex)  # 絶対値の二乗をとると確率になる
        self._amp[0]  = 1                                          # 0番目の状態の確率を1とする
        self._amp     = self._amp.reshape((self.n_states, 1))

    def set_bits(self, bits):
        """
        Arguments:
            bits (list): 状態を表現するリスト ex).|0010> なら [0, 0, 1, 0]
        Returens:
            self (Qubits): 自分自身
        """
        # 例えば |0010> なら idx=2番目(0010番目)となる
        idx            = sum(bit<<i for i, bit in enumerate(bits[::-1]))   # 1の位置をindexにする ( bit<<i = bit*2**i )
        self._amp      = np.zeros(self.n_states, dtype=np.complex)         # 絶対値の二乗をとると確率になる
        self._amp[idx] = 1.0                                               # index番目の確率を1とする
        self._amp     = self._amp.reshape((self.n_states, 1))
        return self

    def measure(self):
        """
        量子ビットを観測する
        後で第kビット目だけを観測するメソッドも作ることにする

        Returens:
            bits (list): 観測された状態を表すリスト
        """
        self._amp     = self._amp.reshape((self.n_states))
        p    = np.abs(self._amp)**2                              # self.amp を観測確率にする
        idx  = np.random.choice(range(len(self._amp)), p=p)      # 適当にindexを決める。pは合計値が1となるlist
        bits = [idx>>i & 1 for i in range(self.n_bits)]         # index番目だけ状態が1となる。その他は0 ( idx>>i = int(idx/2**i) )
        return bits[::-1]                                       # 観測された結果を返す

    def apply(self, *operators):
        """
        量子ビットに演算子を適用する

        Arguments:
            operators (list): 演算子のリスト(何個でも可)
        Returens:
            applied (Qubits): 演算子適用後の量子ビット
        """
        amp = self._amp
        for op in operators:
            amp = op.dot(amp)                  # 演算子を左からかけていく
        applied = self.__class__(self.n_bits)  # 自分のクラスを呼び出している
        applied._amp = amp                     # 演算後の状態を記録する
        return applied                         # 演算後の量子ビットを返す ( applied.__str__ が呼び出される )
        
        
    def __str__(self):
        """
        print時に呼び出されるメソッド
        
        Returns:
            (amp)|0010>
        """
        return " + ".join(
            ("{}|{:0" + str(self.n_bits) + "b}>").format(amp, i)
            for i, amp in enumerate(self._amp) if amp
        )

In [112]:
# Tests
qubits = Qubits(2)                # 2量子ビットを定義
qubits.set_bits([1, 0])  # 1量子ビット目のみ|1>で、他は|0>

print("ビット数",qubits.n_bits)
print("行数",qubits.n_states)

print("重さ的な奴",qubits._amp)
print("10番目(二進数)は",qubits._amp[int('10', 2)])
print("合計は",qubits._amp.sum())

print("量子表示",qubits)
print("観測結果",qubits.measure())

x=np.array([[0,1],
            [1,0]])
qubits = Qubits(1)
qubits.set_bits([0])

ans = qubits.apply(x)
print("演算結果",ans)

ビット数 2
行数 4
重さ的な奴 [[0.+0.j]
 [0.+0.j]
 [1.+0.j]
 [0.+0.j]]
10番目(二進数)は [1.+0.j]
合計は (1+0j)
量子表示 [1.+0.j]|10>
観測結果 [1, 0]
演算結果 [1.+0.j]|1>


# 演算関数

### テンソル積

In [113]:
def tensor(*operater):
    result = operater[0]
    for i in range(1, len(operater)):
        result = np.kron(result, operater[i])
    return result

In [114]:
arr1 = np.array([1,2,3])
arr2 = np.array([1,2])

tensor(arr1, arr2)

array([1, 2, 2, 4, 3, 6])

### 単位演算子 I

In [115]:
def I(n_bits):
    return np.eye(2**n_bits)

In [116]:
I(2)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

### ビット反転演算子 X

In [117]:
def X(n_bits, target):
    return tensor(I(target), np.array([[0,1],[1,0]]), I(n_bits-target-1))

In [118]:
X(2,0) # 第一引数: ビット数  第二引数: 標的ビット

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]])

### アダマールゲート H

In [119]:
def H(n_bits, target):
    return tensor(I(target), np.array([[1,1],[1,-1]])/np.sqrt(2), I(n_bits-target-1))

In [120]:
h = H(1,0)
qubits = Qubits(1)
qubits.set_bits([1])
print(h)
print(qubits)

print(qubits.apply(h))

[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]
[1.+0.j]|1>
[0.70710678+0.j]|0> + [-0.70710678+0.j]|1>


### CNOT演算子

In [131]:
def CNOT(n_bits, control, target):
    pass

### Tゲート

In [132]:
def T(n_bits, target):
    pass

### CUゲート

In [133]:
def CU(U):
    pass